<a href="https://colab.research.google.com/github/Tatsuro0726/chemoinfomatics/blob/main/deepchem/Tutorial6_graphconvolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Tutorial6: Graph Convolutionの紹介
このチュートリアルでは、"グラフ畳み込み "について学びます。これは、分子データを扱うための最も強力な深層学習ツールの一つです。その理由は、分子は自然にグラフとして見ることができるからです

#### Setup

In [2]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  16619      0 --:--:-- --:--:-- --:--:-- 16540


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [3]:
!pip install --pre deepchem

     |████████████████████████████████| 532kB 4.2MB/s 


#### Graph Convolutionとは？
グラフ上で動作します。それらは，グラフの各ノードのデータベクトル（例えば，そのノードが表す原子の化学的性質）から始まります．畳み込み層とプーリング層は、接続されたノード（例えば、互いに結合している原子）からの情報を組み合わせて、各ノードの新しいデータベクトルを生成します。

In [4]:
# Tox21データの読み込み
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train, valid, test = datasets

In [5]:
# 分類モデルの作成
n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks=n_tasks, mode='classification')
model.fit(train, nb_epoch=50)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(436,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(436, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1672,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_16

0.27273563385009764

In [8]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('Training set score:', model.evaluate(train, [metric], transformers))
print('Test set score:', model.evaluate(test, [metric], transformers))

Training set score: {'roc_auc_score': 0.9719191166532942}
Test set score: {'roc_auc_score': 0.7157661835882467}


- GraphConvModelを自身で構築する場合
    - GraphConv Layer: グラフの畳み込みを実装。ノードごとの特徴ベクトルを非線形に隣接ノードの特徴ベクトルと結合する。
    - GraphPool Layer: 近傍の原子の特徴ベクトルの最大値プーリングを計算する。
    - GraphGather: グラフのノードごとに特徴ベクトルを操作。例えば、分子の場合、各ノードは原子を表し、ネットワークは原子の部分的な化学的性質をサマライズした特徴ベクトルを計算する。しかしながら、アプリケーションの最終的な目標としては、分子レベルの特徴表現を適用したい場合がある。このレイヤは、すべてのノードの特徴ベクトルを組み合わせてグラフレベルの特徴ベクトルを作成する。
    - この他にニューラルネットワークの標準的な層(Dense層, BatchNormalization層, Softmax層)が使用できる。

In [19]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100

class MyGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(MyGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

- 2つの畳み込みブロックで構成されている。
    - GraphConv - BatchNorm-Maxpool
- 最終ブロックは以下の構成。
    - Dense - BatchNorm - GraphGather - Dense

In [20]:
# 上で作成したNNモデルのインスタンス生成
model = dc.models.KerasModel(MyGraphConvModel(), loss=dc.models.losses.CategoricalCrossEntropy())

このモデルの入力は何ですか？グラフ畳み込みには、各分子の完全な記述が必要で、ノード（原子）のリストや、どの分子が互いに結合しているかの記述を含みます。実際、データセットを調べてみると、特徴配列にはConvMol型のPythonオブジェクトが含まれていることがわかります

モデルの入力は、Pythonオブジェクトではなく数値配列でなければならない。  
ConvMolオブジェクトをGraphConv, GraphPool, GraphGatherレイヤに対応するデータの形に変換する必要があり、ConvMolクラスにはこの変換が実装されている。

In [21]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import numpy as np

def data_generator(dataset, epochs=1):
    for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs, deterministic=False, pad_batches=True)):
        multiConvMol = ConvMol.agglomerate_mols(X_b)
        inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
        inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
        labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
        weights = [w_b]
    yield (inputs, labels, weights)

In [22]:
model.fit_generator(data_generator(train, epochs=50))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/my_graph_conv_model_1/graph_pool_5/Reshape_14:0", shape=(392,), dtype=int32), values=Tensor("gradient_tape/my_graph_conv_model_1/graph_pool_5/Reshape_13:0", shape=(392, 128), dtype=float32), dense_shape=Tensor("gradient_tape/my_graph_conv_model_1/graph_pool_5/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/my_graph_conv_model_1/graph_pool_5/Reshape_17:0", shape=(1652,), dtype=int32), values=Tensor("gradient_tape/my_graph_conv_model_1/graph_pool_5/Reshape_16:0", shape=(1652, 128), dtype=float32), dense_sha

1.3534796237945557

In [24]:
print('Training set score:', model.evaluate_generator(data_generator(train), [metric], transformers))
print('Test set score:', model.evaluate_generator(data_generator(test), [metric], transformers))

Training set score: {'roc_auc_score': 0.4909027592426832}
Test set score: {'roc_auc_score': 0.4895841408027226}
